<h3><b> Based on this exercise, discuss the the questions in Quizizz with your group

Steps in a user-based recommendation system:

1. Select a user with the movies the user has watched
2. Based on his rating to movies, find the top x neighbours
3. Get the watched movie record of the user for each neighbour.
4. Calculate a similarity score using some formula
5. Recommend the items with the highest score

In [189]:
import pandas as pd
from math import sqrt
import numpy as np


In [190]:
book_df = pd.read_csv('Books.csv')
ratings_df = pd.read_csv('Ratings.csv')
print(book_df.info())



/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271360 entries, 0 to 271359
Data columns (total 8 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   ISBN                 271360 non-null  object
 1   Book-Title           271360 non-null  object
 2   Book-Author          271359 non-null  object
 3   Year-Of-Publication  271360 non-null  object
 4   Publisher            271358 non-null  object
 5   Image-URL-S          271360 non-null  object
 6   Image-URL-M          271360 non-null  object
 7   Image-URL-L          271357 non-null  object
dtypes: object(8)
memory usage: 16.6+ MB
None


In [191]:
userInput = [{'Book-Title':'The Bodyguard\'s Story: Diana, the Crash, and the Sole Survivorn',    'Book-Rating':7.0},
             {'Book-Title':'Sans parler du chien',         'Book-Rating':0.0},
             {'Book-Title':'Life of Pi', 'Book-Rating':9.5},
             {'Book-Title':'Plateforme',                    'Book-Rating':9.0},
             {'Book-Title':'Le chateau des carpathes',      'Book-Rating':8.0}]
inputbook = pd.DataFrame(userInput)
print(inputbook)

                                          Book-Title  Book-Rating
0  The Bodyguard's Story: Diana, the Crash, and t...          7.0
1                               Sans parler du chien          0.0
2                                         Life of Pi          9.5
3                                         Plateforme          9.0
4                           Le chateau des carpathes          8.0


In [192]:
inputId = book_df[book_df['Book-Title'].isin(inputbook['Book-Title'].tolist())]
inputbook = pd.merge(inputId, inputbook)
# inputMovies = inputMovies.drop('year', 1) #we don't really need this at the moment
inputMovies = inputbook[['ISBN','Book-Title','Book-Rating']]
print(inputbook)

         ISBN                Book-Title         Book-Author  \
0  0151008116                Life of Pi         Yann Martel   
1  0156027321                Life of Pi         Yann Martel   
2  1565117794                Life of Pi         Yann Martel   
3  184195425X                Life of Pi         Yann Martel   
4  2290321028  Le chateau des carpathes         Jules Verne   
5  2290324914      Sans parler du chien       Connie Willis   
6  2290321230                Plateforme  Michel Houellebecq   
7  2080682377                Plateforme  Michel Houellebecq   

  Year-Of-Publication         Publisher  \
0                2002          Harcourt   
1                2003     Harvest Books   
2                2003  Highbridge Audio   
3                2004    Pub Group West   
4                2002           J'ai lu   
5                2003           J'ai lu   
6                2003   Distribooks Inc   
7                2001        Flammarion   

                                         Ima

#### Now with the `movieId` in our input, we can now get the subset of users that have watched and reviewed the movies in our input. Find the similar user taste.

In [193]:
userSubset = ratings_df[ratings_df['ISBN'].isin(inputMovies['ISBN'].tolist())]
print(userSubset.groupby('ISBN').count())

            User-ID  Book-Rating
ISBN                            
0151008116      170          170
0156027321      462          462
1565117794       11           11
184195425X       21           21
2080682377        1            1
2290321028        2            2
2290321230        9            9
2290324914        3            3


In [194]:
#Groupby creates several sub dataframes where they all have the same value in the column specified as the parameter
userSubsetGroup = userSubset.groupby(['User-ID'])

def take_5_elem(x):
    # print (len(x[1]))
    return len(x[1])
    

#Sorting it so users with movie most in common with the input will have priority
userSubsetGroup = sorted(userSubsetGroup, key=take_5_elem, reverse=True)

userSubsetGroup = userSubsetGroup[0:100]
print(userSubsetGroup[0:5])


[(11676,        User-ID        ISBN  Book-Rating
46638    11676  0151008116            6
46678    11676  0156027321            6
56002    11676  184195425X            0
56571    11676  2290321230            0), (25436,         User-ID        ISBN  Book-Rating
110097    25436  0156027321            0
110211    25436  2290324914            0), (43246,         User-ID        ISBN  Book-Rating
193912    43246  0151008116            0
193921    43246  0156027321            0), (89602,         User-ID        ISBN  Book-Rating
373240    89602  0151008116            0
373246    89602  0156027321            0), (125519,         User-ID        ISBN  Book-Rating
517843   125519  0156027321            0
518051   125519  1565117794           10)]


In [195]:
#Store the Pearson Correlation in a dictionary, where the key is the user Id and the value is the coefficient
pearsonCorrelationDict = {}

#For every user group in our subset
for name, group in userSubsetGroup:

    #Let's start by sorting the input and current user group so the values aren't mixed up later on
    group = group.sort_values(by='ISBN')
    inputMovies = inputMovies.sort_values(by='ISBN')

    #Get the N for the formula
    nRatings = len(group)

    #Get the review scores for the movies that they both have in common
    temp_df = inputMovies[inputMovies['ISBN'].isin(group['ISBN'].tolist())]

    #And then store them in a temporary buffer variable in a list format to facilitate future calculations
    tempRatingList = temp_df['Book-Rating'].tolist()
   
    #Let's also put the current user group reviews in a list format
    tempGroupList = group['Book-Rating'].tolist()
   
    
    #Now let's calculate the pearson correlation between two users, so called, x and y manually (check the formula from week 7 slide)
    Sxx = sum([i**2 for i in tempRatingList]) - pow(sum(tempRatingList),2)/float(nRatings)
    Syy = sum([i**2 for i in tempGroupList]) - pow(sum(tempGroupList),2)/float(nRatings)
    Sxy = sum( i*j for i, j in zip(tempRatingList, tempGroupList)) - sum(tempRatingList)*sum(tempGroupList)/float(nRatings)

    #If the denominator is different than zero, then divide, else, 0 correlation.
    if Sxx != 0 and Syy != 0:
        pearsonCorrelationDict[name] = Sxy/sqrt(Sxx*Syy)
    else:
        pearsonCorrelationDict[name] = 0
    


In [196]:
pearsonDF = pd.DataFrame.from_dict(pearsonCorrelationDict, orient='index')
pearsonDF.columns = ['similarityIndex']
pearsonDF['User-ID'] = pearsonDF.index
pearsonDF.index = range(len(pearsonDF))
print(pearsonDF.head())

   similarityIndex  User-ID
0          0.57735    11676
1          0.00000    25436
2          0.00000    43246
3          0.00000    89602
4          0.00000   125519


In [197]:
topUsers=pearsonDF.sort_values(by='similarityIndex', ascending=False)[0:50]
print(topUsers.head())

    similarityIndex  User-ID
0           0.57735    11676
63          0.00000    19529
73          0.00000    23768
72          0.00000    22650
71          0.00000    22362


In [198]:
topUsersRating=topUsers.merge(ratings_df, left_on='User-ID', right_on='User-ID', how='inner')
print(topUsersRating.head(100))

    similarityIndex  User-ID            ISBN  Book-Rating
0           0.57735    11676      9022906116            7
1           0.57735    11676  \0432534220\""            6
2           0.57735    11676  \2842053052\""            7
3           0.57735    11676   0 7336 1053 6            0
4           0.57735    11676     0=965044153            7
..              ...      ...             ...          ...
95          0.57735    11676      0006550789            7
96          0.57735    11676      0006551076            6
97          0.57735    11676      000655220X            9
98          0.57735    11676      0006552390            5
99          0.57735    11676      0006552412            5

[100 rows x 4 columns]


In [199]:
#Multiplies the similarity by the user’s ratings
topUsersRating['weightedRating'] = topUsersRating['similarityIndex']*topUsersRating['Book-Rating']
print(topUsersRating.head())

   similarityIndex  User-ID            ISBN  Book-Rating  weightedRating
0          0.57735    11676      9022906116            7        4.041452
1          0.57735    11676  \0432534220\""            6        3.464102
2          0.57735    11676  \2842053052\""            7        4.041452
3          0.57735    11676   0 7336 1053 6            0        0.000000
4          0.57735    11676     0=965044153            7        4.041452


In [200]:
#Applies a sum to the topUsers after grouping it up by movieId
tempTopUsersRating = topUsersRating.groupby('ISBN').sum()[['similarityIndex','weightedRating']]
tempTopUsersRating.columns = ['sum_similarityIndex','sum_weightedRating']
print(tempTopUsersRating.head())

               sum_similarityIndex  sum_weightedRating
ISBN                                                  
 9022906116                0.57735            4.041452
0 7336 1053 6              0.57735            0.000000
0000000000                 0.57735            5.196152
00000000000                0.57735            4.618802
0001055666                 0.57735            5.196152


In [201]:
#Creates an empty dataframe
recommendation_df = pd.DataFrame()

#Now we take the weighted average
recommendation_df['weighted average recommendation score'] = tempTopUsersRating['sum_weightedRating']/tempTopUsersRating['sum_similarityIndex']
recommendation_df['ISBN'] = tempTopUsersRating.index
print(recommendation_df.head(10))

               weighted average recommendation score           ISBN
ISBN                                                               
 9022906116                                      7.0     9022906116
0 7336 1053 6                                    0.0  0 7336 1053 6
0000000000                                       9.0     0000000000
00000000000                                      8.0    00000000000
0001055666                                       9.0     0001055666
0001837397                                       0.0     0001837397
0001841572                                       0.0     0001841572
0001848445                                       0.0     0001848445
0001900277                                       0.0     0001900277
0001941941                                       0.0     0001941941


In [202]:
recommendation_df = recommendation_df.sort_values(by='weighted average recommendation score', ascending=False)
print(recommendation_df)


               weighted average recommendation score           ISBN
ISBN                                                               
0373484232                                      10.0     0373484232
3442438241                                      10.0     3442438241
0812590457                                      10.0     0812590457
0374525641                                      10.0     0374525641
0140444416                                      10.0     0140444416
...                                              ...            ...
B00005WQXK                                       NaN     B00005WQXK
B00005Y01P                                       NaN     B00005Y01P
B00009NDB8                                       NaN     B00009NDB8
N0553212583>>                                    NaN  N0553212583>>
O451203771                                       NaN     O451203771

[25841 rows x 2 columns]


In [203]:
recommended_movie=book_df.loc[book_df['ISBN'].isin(recommendation_df['ISBN'])]

#we don't want to recommend the same movie
recommended_movie=recommended_movie.loc[~recommended_movie.ISBN.isin(userSubset['ISBN'])]

recommended_movie.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
5,0399135782,The Kitchen God's Wife,Amy Tan,1991,Putnam Pub Group,http://images.amazon.com/images/P/0399135782.0...,http://images.amazon.com/images/P/0399135782.0...,http://images.amazon.com/images/P/0399135782.0...
7,0671870432,PLEADING GUILTY,Scott Turow,1993,Audioworks,http://images.amazon.com/images/P/0671870432.0...,http://images.amazon.com/images/P/0671870432.0...,http://images.amazon.com/images/P/0671870432.0...
10,0771074670,Nights Below Station Street,David Adams Richards,1988,Emblem Editions,http://images.amazon.com/images/P/0771074670.0...,http://images.amazon.com/images/P/0771074670.0...,http://images.amazon.com/images/P/0771074670.0...
18,0440234743,The Testament,John Grisham,1999,Dell,http://images.amazon.com/images/P/0440234743.0...,http://images.amazon.com/images/P/0440234743.0...,http://images.amazon.com/images/P/0440234743.0...
